In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%bash
pip install textwrap3
pip install transformers


In [ ]:
df=pd.read_csv('/kaggle/input/twitter-hate-speech/train_E6oV3lV.csv')

# As we can see we have inbalaced dataset,it is leading that majority label would have been predicted much often than other ,which makes overfit on majority class

In [ ]:
df['label'].value_counts()

In [ ]:
df_non_hate=df[df['label']==0]

In [ ]:
df_non_hate=df_non_hate.sample(2242)

In [ ]:
df_hate=df[df['label']==1]

In [ ]:
df_hate=df_hate.append(df_non_hate)

In [ ]:
df_hate.drop('id',axis=1,inplace=True)

In [ ]:
from sklearn.utils import shuffle
df=shuffle(df_hate)

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df

In [ ]:
print('\xF0\x9F\x98\x81')

# Let's remove useless information such as numbers,urls because they are not necessary while we are using language model such as Bert

In [ ]:
import re

def clean(text):
    text=re.sub('@user','',text)
    text=re.sub('\d','',text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return text

In [ ]:
df['tweet']=df['tweet'].apply(lambda x:clean(x))

In [ ]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup,RobertaForSequenceClassification,RobertaConfig
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# run everything on cuda(if exists) or on cpu 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
# MODEL_CLASSES = {
#     'roberta': (RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig)
# }

#  Set some hyper parameters our model and parameters for tokenizer and make data with lower case and truncation

In [ ]:
MAX_LEN = 140
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)


# Get  data labels,tweets and tokenized data 

In [ ]:
class Data(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.tweet
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_size = 0.8
train_data=df.sample(frac=train_size,random_state=200)
test_data=df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = Data(train_data, tokenizer, MAX_LEN)
testing_set = Data(test_data, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# set our model with roberta and adding some more layers to preprocess embeddings

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)
        self.res=torch.nn.Softmax()

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        res=self.res(output)
        return res

In [ ]:
# class RobertaClassnew(torch.nn.Module):
#     def __init__(self):
#         super(RobertaClassnew, self).__init__()
#         self.l1 = RobertaModel.from_pretrained("roberta-base")

#         self.classifier = torch.nn.Linear(768, 5)

#     def forward(self, input_ids, attention_mask, token_type_ids):
#         output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#         hidden_state = output_1[0]
#         pooler = hidden_state[:, 0]
#         output = self.classifier(pooler)
#         return output

# model_fn for fine tune and other set for without

In [ ]:
model_fn = RobertaClass()
model_fn.to(device)

In [ ]:
model = RobertaClass()
model.to(device)

# set loss functions and optimizers for our fine-tuning network

In [ ]:
loss_function_fn = torch.nn.CrossEntropyLoss()
optimizer_fn = torch.optim.Adam(params =  model_fn.parameters(), lr=LEARNING_RATE)


In [ ]:
loss_function = torch.nn.CrossEntropyLoss()



In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct


In [ ]:
# for param in model.l1.parameters():
    
#     param.requires_grad=False

# let's make fine tune getting embeddings and training

In [ ]:
def train_fn(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model_fn.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        #print(data)
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        
        outputs = model_fn(ids, mask, token_type_ids)
        loss = loss_function_fn(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer_fn.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer_fn.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [ ]:
from tqdm import tqdm
EPOCHS = 1
for epoch in range(EPOCHS):
    train_fn(epoch)

# let's validate our models and getting from them weight values

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    collect_pred=[]
    collect_target=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            
            loss = loss_function_fn(outputs, targets)
            
            tr_loss += loss.item()
            
            big_val, big_idx = torch.max(outputs.data, dim=1)
            #print(outputs.data)
            n_correct += calcuate_accuracy(big_idx, targets)
           
            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            collect_pred.append(outputs.cpu().detach().numpy())
            collect_target.append(big_idx.cpu().detach().numpy())
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return collect_pred,collect_target,epoch_accu

In [ ]:
pred_fn,tar_fn,acc_fn = valid(model_fn, testing_loader)
print("fine tune accuracy on test data = %0.2f%%" % acc_fn)

# 

In [ ]:
pred,tar,acc = valid(model, testing_loader)
print("without fine tune accuracy on test data = %0.2f%%" % acc)

# fine-tuning gives for us better results and let's preparing data for the graphs and plotting pr-curve

In [ ]:
pred=pd.DataFrame(np.concatenate(np.array(pred)))
pred_fn=pd.DataFrame(np.concatenate(np.array(pred_fn)))
tar=pd.DataFrame(np.concatenate(np.array(tar)))
tar_fn=pd.DataFrame(np.concatenate(np.array(tar_fn)))

In [ ]:
tar_fn

In [ ]:
output_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)


In [ ]:
pred_fn

In [ ]:
# pred.reshape(-1)

In [ ]:
pred

In [ ]:
from sklearn.metrics import precision_recall_curve,auc
precision_fn, recall_fn, thresholds_fn =precision_recall_curve(test_data['label'].values,pred_fn[1].values)

In [ ]:
precision, recall, thresholds =precision_recall_curve(test_data['label'].values,pred[1].values)

In [ ]:
# pred_fn['res']=pred_fn.apply(lambda x: 1 if x[1]>x[0] else 0,axis=1)

In [ ]:
plt.plot(recall_fn,precision_fn,marker='.')
plt.xlabel('Recall fine_tune')
plt.ylabel('Precision fine_tune')

In [ ]:
plt.plot(recall,precision,marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')

In [ ]:
import seaborn as sns
sns.distplot(pred_fn[0])

In [ ]:
import seaborn as sns
sns.distplot(pred[0])

In [ ]:
import seaborn as sns
sns.distplot(pred[1])

In [ ]:
import seaborn as sns
sns.distplot(pred_fn[1])

In [ ]:
auc(recall,precision)

# As we can see  most of weights have frequent values nearby 0.00 and 1.00 this means that we should some how standartize weights as we can.However, as for me it is better to apply on hate speech detection,because accuracy is better for detecting the whole classes 